In [1]:
pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.0/287.0 kB 6.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from google.cloud import bigquery

In [3]:
# Initialize BigQuery client
client = bigquery.Client()

In [ ]:
from nba_api.stats.endpoints import LeagueGameLog, BoxScoreTraditionalV2
from google.cloud import bigquery
import pandas as pd
from tqdm import tqdm
from requests import Session
import time
from time import sleep
from requests.exceptions import ReadTimeout

# BigQuery setup
client = bigquery.Client()
dataset_id = 'fantasy-survivor-app.nba_data'
table_id = f'{dataset_id}.player_historical_game_stats'

# Define schema for BigQuery
schema = [
    bigquery.SchemaField('season', 'STRING'),
    bigquery.SchemaField('game_id', 'INT64'),
    bigquery.SchemaField('team_id', 'INT64'),
    bigquery.SchemaField('team_abbreviation', 'STRING'),
    bigquery.SchemaField('player_id', 'INT64'),
    bigquery.SchemaField('player_name', 'STRING'),
    bigquery.SchemaField('start_position', 'STRING'),
    bigquery.SchemaField('comment', 'STRING'),
    bigquery.SchemaField('minute', 'FLOAT64'),
    bigquery.SchemaField('fg_made', 'INT64'),
    bigquery.SchemaField('fg_attempts', 'INT64'),
    bigquery.SchemaField('fg_pct', 'FLOAT64'),
    bigquery.SchemaField('three_p_made', 'INT64'),
    bigquery.SchemaField('three_p_attempts', 'INT64'),
    bigquery.SchemaField('three_p_pct', 'FLOAT64'),
    bigquery.SchemaField('ft_made', 'INT64'),
    bigquery.SchemaField('ft_attempts', 'INT64'),
    bigquery.SchemaField('ft_pct', 'FLOAT64'),
    bigquery.SchemaField('offensive_rebounds', 'INT64'),
    bigquery.SchemaField('defensive_rebounds', 'INT64'),
    bigquery.SchemaField('rebounds', 'INT64'),
    bigquery.SchemaField('assists', 'INT64'),
    bigquery.SchemaField('steals', 'INT64'),
    bigquery.SchemaField('blocks', 'INT64'),
    bigquery.SchemaField('turnovers', 'INT64'),
    bigquery.SchemaField('personal_fouls', 'INT64'),
    bigquery.SchemaField('points', 'INT64'),
    bigquery.SchemaField('plus_minus', 'FLOAT64'),
    bigquery.SchemaField('double_double', 'BOOL'),
    bigquery.SchemaField('triple_double', 'BOOL')

]

job_config = bigquery.LoadJobConfig(schema=schema, write_disposition='WRITE_APPEND')

# Function to calculate double-double and triple-double
def calculate_double_triple_double(stats):
    points, rebounds, assists, blocks, steals = stats['PTS'], stats['REB'], stats['AST'], stats['BLK'], stats['STL']
    double_double = (
        sum(x >= 10 for x in [points, rebounds, assists, blocks, steals]) >= 2
    )
    triple_double = (
        sum(x >= 10 for x in [points, rebounds, assists, blocks, steals]) >= 3
    )
    return double_double, triple_double

# Fetch stats for a single game
def fetch_stats_for_game(game_id, season, max_retries=3, timeout=30):
    attempt = 0
    while attempt < max_retries:
        try:
            # Fetch data from the API
            boxscore = BoxScoreTraditionalV2(game_id=game_id, timeout=timeout)
            headers = boxscore.get_dict()['resultSets'][0]['headers']
            rows = boxscore.get_dict()['resultSets'][0]['rowSet']
            game_df = pd.DataFrame(rows, columns=headers)

            if game_df.empty:
                return pd.DataFrame()

            # Add season column
            game_df['SEASON'] = season
            game_df['GAME_ID'] = game_df['GAME_ID'].str.lstrip('0').astype('int64')
            game_df['TEAM_ID'] = game_df['TEAM_ID'].astype('int64')
            game_df['PLAYER_ID'] = game_df['PLAYER_ID'].astype('int64')

            # Process MIN column: Extract only the integer part of the time
            game_df['MIN'] = game_df['MIN'].apply(
                lambda x: int(float(x.split(":")[0])) if pd.notnull(x) and isinstance(x, str) else None
            )

            # Calculate double-double and triple-double
            game_df['DOUBLE_DOUBLE'], game_df['TRIPLE_DOUBLE'] = zip(
                *game_df.apply(lambda x: calculate_double_triple_double(x), axis=1)
            )

            # Select and rename columns
            game_df = game_df[[
                'SEASON', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME',
                'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
                'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
                'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS', 'DOUBLE_DOUBLE', 'TRIPLE_DOUBLE'
            ]]

            # Define the column mapping based on the schema
            column_mapping = {
                'SEASON': 'season',
                'GAME_ID': 'game_id',
                'TEAM_ID': 'team_id',
                'TEAM_ABBREVIATION': 'team_abbreviation',
                'PLAYER_ID': 'player_id',
                'PLAYER_NAME': 'player_name',
                'START_POSITION': 'start_position',
                'COMMENT': 'comment',
                'MIN': 'minute',
                'FGM': 'fg_made',
                'FGA': 'fg_attempts',
                'FG_PCT': 'fg_pct',
                'FG3M': 'three_p_made',
                'FG3A': 'three_p_attempts',
                'FG3_PCT': 'three_p_pct',
                'FTM': 'ft_made',
                'FTA': 'ft_attempts',
                'FT_PCT': 'ft_pct',
                'OREB': 'offensive_rebounds',
                'DREB': 'defensive_rebounds',
                'REB': 'rebounds',
                'AST': 'assists',
                'STL': 'steals',
                'BLK': 'blocks',
                'TO': 'turnovers',
                'PF': 'personal_fouls',
                'PTS': 'points',
                'PLUS_MINUS': 'plus_minus',
                'DOUBLE_DOUBLE': 'double_double',
                'TRIPLE_DOUBLE': 'triple_double'
            }

            # Rename columns in the DataFrame using the mapping
            game_df.rename(columns=column_mapping, inplace=True)

            return game_df

        except ReadTimeout:
            attempt += 1
            print(f"Timeout occurred for game_id {game_id}. Retrying {attempt}/{max_retries}...")
            sleep(5)  # Wait before retrying

        except Exception as e:
            print(f"Error fetching data for game_id {game_id}: {e}")
            return pd.DataFrame()

    # If retries are exhausted, raise an exception or return an empty DataFrame
    print(f"Failed to fetch data for game_id {game_id} after {max_retries} retries.")
    return pd.DataFrame()


# Get all game IDs for the past three seasons
def get_game_ids_for_season(season):
    game_log = LeagueGameLog(season=season, season_type_all_star='Regular Season')
    games = game_log.get_data_frames()[0]
    return games['GAME_ID'].unique().tolist()

# Process and save historical data
#seasons = ["2021-22", "2022-23", "2023-24"]
seasons = ["2024-25"]
batch_size = 100
batch = []

for season in seasons:
    print(f"Processing season: {season}")
    game_ids = get_game_ids_for_season(season)

    for i, game_id in enumerate(tqdm(game_ids, desc=f"Processing games for {season}")):
        game_stats = fetch_stats_for_game(game_id, season)
        if not game_stats.empty:
            batch.append(game_stats)

        # Save to BigQuery in batches
        if len(batch) >= batch_size or i == len(game_ids) - 1:
            batch_df = pd.concat(batch, ignore_index=True)
            job = client.load_table_from_dataframe(batch_df, table_id, job_config=job_config)
            job.result()  # Wait for the job to complete
            print(f"Saved {len(batch_df)} rows to BigQuery.")
            batch = []  # Clear the batch


Processing season: 2024-25


Processing games for 2024-25:   1%|          | 14/1230 [00:01<02:33,  7.91it/s]

Timeout occurred for game_id 0022400076. Retrying 1/3...
Timeout occurred for game_id 0022400076. Retrying 2/3...
Timeout occurred for game_id 0022400076. Retrying 3/3...


Processing games for 2024-25:   1%|▏         | 17/1230 [01:47<4:38:06, 13.76s/it]

Failed to fetch data for game_id 0022400076 after 3 retries.


...

Processing games for 2024-25:  54%|█████▍    | 663/1230 [1:43:50<01:48,  5.21it/s]

Timeout occurred for game_id 0022400644. Retrying 1/3...
Timeout occurred for game_id 0022400644. Retrying 2/3...
Timeout occurred for game_id 0022400644. Retrying 3/3...


Processing games for 2024-25:  54%|█████▍    | 665/1230 [1:45:16<2:38:03, 16.78s/it]

Failed to fetch data for game_id 0022400644 after 3 retries.


Processing games for 2024-25:  55%|█████▌    | 679/1230 [1:45:17<09:26,  1.03s/it]

In [ ]:
# Query Relevant Table
df_query = """
SELECT * FROM `fantasy-survivor-app.nba_data.game_log` limit 100
"""

df_job= client.query(
df_query,
location="northamerica-northeast1",
)
df_raw = df_job.to_dataframe()

NotFound: 404 Not found: Table fantasy-survivor-app:nba_data.game_log was not found in location northamerica-northeast1; reason: notFound, message: Not found: Table fantasy-survivor-app:nba_data.game_log was not found in location northamerica-northeast1

Location: northamerica-northeast1
Job ID: 982922b6-17b9-4fd8-a7c7-0de16877c10f


In [ ]:
df_raw.head()

,game_id,team_id,team_abbreviation,player_id,player_name,start_position,comment,minute,fg_made,fg_attempts,...,assists,steals,blocks,turnovers,personal_fouls,points,plus_minus,double_double,triple_double,season
0,22300015,1610612756,PHX,1629626,Bol Bol,,DNP - Coach's Decision,NaN,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,False,2023-24
1,22300185,1610612756,PHX,1629626,Bol Bol,,DNP - Coach's Decision,NaN,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,False,2023-24
2,22300197,1610612756,PHX,1629626,Bol Bol,,DNP - Coach's Decision,NaN,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,False,2023-24
3,22300035,1610612756,PHX,1629626,Bol Bol,,DNP - Coach's Decision,NaN,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,False,2023-24
4,22300214,1610612756,PHX,1629626,Bol Bol,,DNP - Coach's Decision,NaN,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,False,False,2023-24


In [ ]:
from nba_api.stats.endpoints import LeagueGameLog
import pandas as pd
import datetime
import time
from google.cloud import bigquery
from requests.exceptions import ReadTimeout

# Retry wrapper for single day pulls
def safe_day_game_log(date_str, season, max_retries=3):
    for attempt in range(max_retries):
        try:
            log = LeagueGameLog(
                season=season,
                season_type_all_star="Regular Season",
                player_or_team_abbreviation="T",
                date_from_nullable=date_str,
                date_to_nullable=date_str,
                timeout=10
            )
            return log.get_data_frames()[0]
        except ReadTimeout:
            print(f"Timeout on {date_str}, retry {attempt+1}/{max_retries}")
            time.sleep(2 * (attempt + 1))
    return pd.DataFrame()

# Helper to generate all dates in a season range
def generate_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
    return [(start + datetime.timedelta(days=i)).strftime("%m/%d/%Y")
            for i in range((end - start).days + 1)]

# Seasons and their date ranges
season_ranges = {
    "2021-22": ("2021-10-01", "2022-06-30"),
    "2022-23": ("2022-10-01", "2023-06-30"),
    "2023-24": ("2023-10-01", "2024-06-30"),
}

all_frames = []

for season, (start, end) in season_ranges.items():
    print(f"Fetching {season}...")
    dates = generate_dates(start, end)
    for d in dates:
        df = safe_day_game_log(d, season)
        if not df.empty:
            df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"]).dt.date
            df["SEASON"] = season
            all_frames.append(df[["GAME_ID", "GAME_DATE", "SEASON"]].drop_duplicates())
        time.sleep(0.5)  # be polite

# Combine results
game_date_map = pd.concat(all_frames, ignore_index=True)

# Save locally (backup)
game_date_map.to_csv("game_log_all.csv", index=False)

# Upload to BigQuery
client = bigquery.Client()
table_id = "fantasy-survivor-app.nba_data.game_log"

job = client.load_table_from_dataframe(
    game_date_map,
    table_id,
    job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
)
job.result()

print(f"Uploaded {len(game_date_map)} rows into {table_id}")


Fetching 2021-22...
Timeout on 10/01/2021, retry 1/3
Timeout on 10/01/2021, retry 2/3
Timeout on 10/01/2021, retry 3/3
Timeout on 10/02/2021, retry 1/3
Timeout on 10/02/2021, retry 2/3
Timeout on 10/02/2021, retry 3/3


KeyboardInterrupt: 